<a href="https://colab.research.google.com/github/AntiVini/QuotationUFRJ/blob/main/fornecedor17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def pesquisa_google(query, item):

  # Passo 1:  

  import requests, json, numpy as np, pandas as pd
  from pandas import DataFrame

  # set up the request parameters
  params = {
    'api_key': '6DD31194DE894AB6B41C7ED1F9761A8E',
    'q': query,
    'search_type': 'shopping',
    'output': 'json',
    'num': '100',
    'gl': 'br',
    'hl': 'pt',
    'google_domain': 'google.com.br',
    'location': 'State of Rio de Janeiro,Brazil'
  }

  # make the http GET request to Scale SERP
  result = requests.get('https://api.scaleserp.com/search', params).json()
  size = len(result['shopping_results'])
    
  # Passos 2 e 3:

  merchant_list = list()
  link_list = list()
  price_list = list()
  merchant_link_list = list()
  indice_link = list()
  store_id = list()
  id_list = list()

  indice = 0
  while len(price_list) < 5:

    if 'merchant' in result['shopping_results'][indice].keys():

      if result['shopping_results'][indice]['merchant'] != 'Mercado Livre' and result['shopping_results'][indice]['merchant'] not in merchant_list:
        merchant = result['shopping_results'][indice]['merchant']
        indice_link.append(result['shopping_results'][indice]['link'])
        store_id.append(result['shopping_results'][indice]['id'])
      
        if (result['shopping_results'][indice]['price_parsed']['value'] > result['shopping_results'][indice]['price']):
          price = result['shopping_results'][indice]['price_parsed']['value']
        else:
          price = result['shopping_results'][indice]['price']
        
        merchant_list.append(merchant)
        price_list.append(price)

      indice += 1

  max = np.average(price_list) + np.std(price_list)
  min = np.average(price_list) - np.std(price_list)  

  for i in range(len(price_list)):
    
    if price_list[i] < min or price_list[i] > max:
      price_list[i] = 0
      merchant_list[i] = 0
      indice_link[i] = 0
      store_id[i] = 0

  while 0 in price_list and indice <= size:
    position = price_list.index(0)

    if result['shopping_results'][indice]['merchant']: 

      if result['shopping_results'][indice]['merchant'] != 'Mercado Livre' and result['shopping_results'][indice]['merchant'] not in merchant_list:
        merchant = result['shopping_results'][indice]['merchant']
        link_position = result['shopping_results'][indice]['link']
        id_new = result['shopping_results'][indice]['id']

        if (result['shopping_results'][indice]['price_parsed']['value'] > result['shopping_results'][indice]['price']):
          price = result['shopping_results'][indice]['price_parsed']['value']
        else:
          price = result['shopping_results'][indice]['price']

        if price <= max and price >= min:
          merchant_list[position] = merchant
          price_list[position] = price
          indice_link[position] = link_position
          store_id[position] = id_new

      indice += 1

  for i in range(size):

    if result['shopping_results'][i]['merchant']:

      if (result['shopping_results'][i]['merchant'] != 'Mercado Livre' and result['shopping_results'][i]['merchant'] not in merchant_link_list):
        link = result['shopping_results'][i]['link']
        merchant_link = result['shopping_results'][i]['merchant']
        id_total = result['shopping_results'][i]['id']
              
        merchant_link_list.append(merchant_link)       
        link_list.append(link)
        id_list.append(id_total)
      
  values = [merchant_list, price_list, link_list]
  item_list = ['-' for i in range(len(link_list))]

  for i in range(len(id_list)):
    
    for j in range(len(store_id)):
      
      if id_list[i] == store_id[j]:
        item_list[i] = j + 1

  # Passo 4:
  
  mapa = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Cópia de PP - Anexo V.xlsx')
  mapa = mapa.drop(['Unnamed: 0'], axis=1)

  for i in range(len(values[1])):
    mapa[f'Unnamed: {i+10}'][2 + 4*(item-1)] = f'SITE: {values[0][i]}'
    mapa[f'Unnamed: {i+10}'][4 + 4*(item-1)] = values[1][i]

  df1 = pd.DataFrame({'links': values[2]})
  df2 = pd.DataFrame({'stores': item_list})
  df_total = df1.join(df2)

  # Passo 5:

  mapa.to_excel(r'/content/drive/MyDrive/Colab Notebooks/Cópia de PP - Anexo V.xlsx')
  df_total.to_excel(f'/content/drive/MyDrive/Colab Notebooks/list_links_item_{item}.xlsx')